#Dataset 1

In [ ]:
import numpy as np
import pandas as pd
from sklearn import metrics
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler

In [ ]:
!pip install -q convokit

In [ ]:
from convokit import Corpus, download
corpus = Corpus(filename=download("friends-corpus"))

corpus.print_summary_stats()
data2 = corpus.get_utterances_dataframe()


data1 = data2[['text', 'speaker']]
data1 = data1.reset_index()
data1 = data1.drop('id',axis=1)


np.savetxt(r'test.txt', data1.values, fmt='%s')
print(data1)

Dataset already exists at /root/.convokit/downloads/friends-corpus
Number of Speakers: 700
Number of Utterances: 67373
Number of Conversations: 3107
                                                    text          speaker
0      There's nothing to tell! He's just some guy I ...    Monica Geller
1      C'mon, you're going out with the guy! There's ...   Joey Tribbiani
2      All right Joey, be nice. So does he have a hum...    Chandler Bing
3                               Wait, does he eat chalk?    Phoebe Buffay
4                                                         TRANSCRIPT_NOTE
...                                                  ...              ...
67368                            Oh, it's gonna be okay.    Chandler Bing
67369  Do you guys have to go to the new house right ...     Rachel Green
67370                                  We got some time.    Monica Geller
67371                   Okay, should we get some coffee?     Rachel Green
67372                                

In [ ]:
for i in range(len(data1)):
    if(data1.iloc[i][-1] == 'Chandler Bing'):
        data1.loc[i][-1] = 'c'
    elif(data1.iloc[i][-1] == 'Rachel Green'):
        data1.loc[i][-1] = 'r'
    elif(data1.iloc[i][-1] == 'Ross Geller'):
        data1.loc[i][-1] = 'R'
    elif(data1.iloc[i][-1] == 'Monica Geller'):
        data1.loc[i][-1] = 'm'
    elif(data1.iloc[i][-1] == 'Phoebe Buffay'):
        data1.loc[i][-1] = 'p'
    elif(data1.iloc[i][-1] == 'Joey Tribbiani'):
        data1.loc[i][-1] = 'j'
    else:
        data1.loc[i][-1] = 'o'

data1.sample(frac=1) 
data1.head()



,text,speaker
0,There's nothing to tell! He's just some guy I ...,m
1,"C'mon, you're going out with the guy! There's ...",j
2,"All right Joey, be nice. So does he have a hum...",c
3,"Wait, does he eat chalk?",p
4,,o


In [ ]:

data1 = data1[data1['speaker'] != 'o']
print(data1.count())
print(data1)


text       51312
speaker    51312
dtype: int64
                                                    text speaker
0      There's nothing to tell! He's just some guy I ...       m
1      C'mon, you're going out with the guy! There's ...       j
2      All right Joey, be nice. So does he have a hum...       c
3                               Wait, does he eat chalk?       p
5      Just, 'cause, I don't want her to go through w...       p
...                                                  ...     ...
67368                            Oh, it's gonna be okay.       c
67369  Do you guys have to go to the new house right ...       r
67370                                  We got some time.       m
67371                   Okay, should we get some coffee?       r
67372                                       Sure. Where?       c

[51312 rows x 2 columns]


In [ ]:
data1 = data1.reset_index(drop=True)
print(data1)

                                                    text speaker
0      There's nothing to tell! He's just some guy I ...       m
1      C'mon, you're going out with the guy! There's ...       j
2      All right Joey, be nice. So does he have a hum...       c
3                               Wait, does he eat chalk?       p
4      Just, 'cause, I don't want her to go through w...       p
...                                                  ...     ...
51307                            Oh, it's gonna be okay.       c
51308  Do you guys have to go to the new house right ...       r
51309                                  We got some time.       m
51310                   Okay, should we get some coffee?       r
51311                                       Sure. Where?       c

[51312 rows x 2 columns]


#LSTM - nivel de caracter


In [ ]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt

import torch
import string
import random
import re
import time, math

import torch.utils.data as utils
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import seaborn as sn
from sklearn.metrics import confusion_matrix

import torchvision
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import collections

import torch.nn as nn
from torch.autograd import Variable
import torch
import torch.utils.data as utils
import csv

In [ ]:
datat = data1[50001:]
print(datat)

                                                    text speaker
50001          Yeah, you miss alot, when you're moo-ing.       m
50002  Oh! It was our pleasure. We are so much enjoyi...       m
50003  Oh, uhm, okay, uhm, do you mind if we ask you ...       c
50004                                      That's great.       c
50005                                    How's that now?       m
...                                                  ...     ...
51307                            Oh, it's gonna be okay.       c
51308  Do you guys have to go to the new house right ...       r
51309                                  We got some time.       m
51310                   Okay, should we get some coffee?       r
51311                                       Sure. Where?       c

[1311 rows x 2 columns]


In [ ]:
data1 = data1[:5000]
print(data1)

                                                   text speaker
0     There's nothing to tell! He's just some guy I ...       m
1     C'mon, you're going out with the guy! There's ...       j
2     All right Joey, be nice. So does he have a hum...       c
3                              Wait, does he eat chalk?       p
4     Just, 'cause, I don't want her to go through w...       p
...                                                 ...     ...
4995                  You gotta tell Ross how you feel.       j
4996  Come on. How can I just tell him? What about J...       r
4997  What about her? They've only been going out fo...       j
4998                        I don't know, I don't know.       r
4999  Look, Rach, Rach! I've been with my share of w...       j

[5000 rows x 2 columns]


In [ ]:
chunk_len = 150

# Gera um pedaço aleatório de texto com o tamanho especificado em chuck_len
def random_chunk():
    index = random.randint(0, len(data1)-1)
    
    data = data1.iloc[index][0]
    
    for i in range(200-len(data)):#normaliza os chunks com tamanho igual
        data+='/0'
        
    return data, index
    
print(random_chunk())

('Does he know?/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0', 3306)


In [ ]:
file = open('test.txt').read()
all_characters = ''.join(set(file))
n_characters = len(all_characters)
print(all_characters)
print(n_characters)
file_len = len(file)
print('file_len =', file_len)




6ztL4vuP,#xER[]nl${VWK?J%+o>i&QjsghUrDH(NwT'`cXY/}Id3pqGkmZ0M1F:f52)
987;!-b_ Ca".OyB*eSA
89
file_len = 4169043


In [ ]:
def char_tensor(string):
    tensor = torch.zeros(len(string)).long()
    for c in range(len(string)):
        try:
            tensor[c] = all_characters.index(string[c])
        except:
            print(c)
            raise
    return Variable(tensor)

print(char_tensor('abcDEF'))

tensor([79, 75, 45, 37, 11, 62])


In [ ]:
# Gera um 'exemplo' aleatório (um pedaço aleatório convertido para lista de inteiros)
def random_training_set():    
    chunk, index = random_chunk()
    inp = char_tensor(chunk[:-1])
    target = data1.iloc[index][-1]
    target = char_tensor(target)
    return inp, target

print(random_training_set())
#tensor = random_training_set()
#print(tensor)
#if(tensor[0][199] == 3) :
   # print("sim!")

(tensor([50, 77, 79, 57, 77, 15, 26,  2, 77, 79, 77, 75, 79, 75, 83, 73, 77, 47,
        26,  6, 77, 56, 15, 26, 41, 77, 41, 34, 79,  2, 22, 77, 50, 77, 32, 41,
        86, 79, 36, 77,  2, 26, 77, 33, 26, 51,  8, 77, 31,  6, 32,  2, 77, 75,
        86, 45, 79,  6, 32, 86, 77, 83, 26,  6, 77, 33, 86,  2, 77, 32, 26, 77,
         6, 53,  2, 28, 33, 34,  2, 77, 86,  5, 86, 36, 83, 77,  2, 28, 57, 86,
        77, 41, 86, 81, 81, 81, 48, 59, 48, 59, 48, 59, 48, 59, 48, 59, 48, 59,
        48, 59, 48, 59, 48, 59, 48, 59, 48, 59, 48, 59, 48, 59, 48, 59, 48, 59,
        48, 59, 48, 59, 48, 59, 48, 59, 48, 59, 48, 59, 48, 59, 48, 59, 48, 59,
        48, 59, 48, 59, 48, 59, 48, 59, 48, 59, 48, 59, 48, 59, 48, 59, 48, 59,
        48, 59, 48, 59, 48, 59, 48, 59, 48, 59, 48, 59, 48, 59, 48, 59, 48, 59,
        48, 59, 48, 59, 48, 59, 48, 59, 48, 59, 48, 59, 48, 59, 48, 59, 48, 59,
        48, 59, 48, 59, 48, 59, 48, 59, 48, 59, 48, 59, 48, 59, 48, 59, 48, 59,
        48, 59, 48, 59, 48, 59, 48, 59,

In [ ]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, n_layers=1):
        super(RNN, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        
        self.encoder = nn.Embedding(input_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size, n_layers)
        self.decoder = nn.Linear(hidden_size, output_size)
    
    def forward(self, input, hidden, hidden2):
        input = self.encoder(input.view(1, -1))
        output, (hidden, hidden2) = self.lstm(input.view(1, 1, -1), (hidden, hidden2))
        decoded_output = self.decoder(output.view(1, -1))
        return decoded_output, hidden, hidden2

    def init_hidden(self):
        return Variable(torch.zeros(self.n_layers, 1, self.hidden_size))

In [ ]:
# Gera um texto iniciando com "prime_str" de tamanho "predict_len" e variação definida por "temperature"
def evaluate(prime_str='A', predict_len=1, temperature=0.8):
    hidden = rnn.init_hidden().cuda()
    hidden2 = rnn.init_hidden().cuda()
    prime_input = char_tensor(prime_str).cuda()
    predicted = ''

    # prime_str é o texto inicial que o gerado irá completar
    for p in range(len(prime_str) - 1):
        _, hidden, hidden2 = rnn(prime_input[p], hidden, hidden2)
    inp = prime_input[-1]
    
    for p in range(predict_len):
        output, hidden, hidden2 = rnn(inp, hidden, hidden2)
        
        # Usa a temperatura para amostrar a distribuição e escolher a saída probabilísticamente
        output_dist = output.data.view(-1).div(temperature).exp()
        top_i = torch.multinomial(output_dist, 1)[0]
        
        # Adiciona o caracter predito à string de saída
        predicted_char = all_characters[top_i]
        predicted += predicted_char
        inp = char_tensor(predicted_char).cuda()

    return predicted

In [ ]:
# Treina sobre um exemplo (i.e. uma amostragem do texto)

def train(inp, target):
   
    hidden = rnn.init_hidden()
    hidden2 = rnn.init_hidden()

    rnn.zero_grad()
    loss = 0

    inp = inp.cuda()
    hidden = hidden.cuda()
    hidden2 = hidden2.cuda()
    for c in range(chunk_len):
        if(inp[c] != 79 or inp[c] != 74):
            output, hidden, hidden2 = rnn(inp[c], hidden, hidden2)
            loss += loss_metric(output, target[0].unsqueeze(0))
        else:
            break

    loss.backward()
    optimizer.step()

    return (loss.data.item() / chunk_len), hidden, hidden2

In [ ]:

def time_since(since):
    s = time.time() - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

In [ ]:
n_epochs = 100
print_every = 1
hidden_size = 512
n_layers = 4
lr = 0.0005

cuda = torch.device('cuda')

rnn = RNN(n_characters, hidden_size, n_characters, n_layers)
rnn.cuda()


optimizer = torch.optim.Adam(rnn.parameters(), lr=lr)
loss_metric = nn.CrossEntropyLoss()
h1 = 0
h2 = 0
start = time.time()
all_losses = []
loss_avg = 0
for epoch in range(1, n_epochs + 1):
    example = random_training_set()
    #print(example[0])
    loss, h1, h2 = train(example[0].cuda(),example[1].cuda())       
    loss_avg += loss

    if epoch % print_every == 0:
        print('[%s (%d %d%%) %.4f]' % (time_since(start), epoch, epoch / n_epochs * 100, loss))
       # print(evaluate('The', 100), '\n')

 0s (1 1%) 4.4641]
 0s (2 2%) 4.3351]
 0s (3 3%) 4.4744]
 1s (4 4%) 4.4942]
 1s (5 5%) 4.5410]
 1s (6 6%) 3.9702]
 2s (7 7%) 4.5203]
 2s (8 8%) 4.4958]
 2s (9 9%) 3.1066]
 3s (10 10%) 3.3943]
 3s (11 11%) 2.5331]
 3s (12 12%) 3.5619]
 3s (13 13%) 3.6429]
 4s (14 14%) 1.6440]
 4s (15 15%) 1.7144]
 4s (16 16%) 3.0342]
 5s (17 17%) 1.4784]
 5s (18 18%) 2.5386]
 5s (19 19%) 4.0982]
 6s (20 20%) 1.5183]
 6s (21 21%) 2.0957]
 6s (22 22%) 1.8874]
 7s (23 23%) 2.1648]
 7s (24 24%) 2.0340]
 7s (25 25%) 3.5304]
 7s (26 26%) 1.8438]
 8s (27 27%) 2.0491]
 8s (28 28%) 1.1495]
 8s (29 28%) 1.1142]
 9s (30 30%) 2.9759]
 9s (31 31%) 0.9795]
 9s (32 32%) 2.1708]
 10s (33 33%) 2.2847]
 10s (34 34%) 1.7779]
 10s (35 35%) 2.0212]
 10s (36 36%) 0.9381]
 11s (37 37%) 2.2986]
 11s (38 38%) 1.7884]
 11s (39 39%) 3.8757]
 12s (40 40%) 1.6682]
 12s (41 41%) 3.6517]
 12s (42 42%) 1.6882]
 13s (43 43%) 1.5072]
 13s (44 44%) 1.3892]
 13s (45 45%) 1.3182]
 13s (46 46%) 1.8640]
 14s (47 47%) 1.8075]
 14s (48 48%) 1.

In [ ]:
#usando o evaluate para avaliar o modelo com o dataset de test
datat = datat[datat['text'].map(len) > 15]

correct = 0
loss = 0

for i in range(len(datat)):
    predict = evaluate(datat.iloc[i][0])
    label = datat.iloc[i][-1]
    if(predict == label):
        correct+=1
    
        
        
        
accuracy = correct/(len(datat))



print('Acurácia: %d%%'  % (accuracy * 100))

Acurácia: 17%


##Teste LSTM caracter

In [ ]:
# carregar o dataset de test

In [ ]:
print(h1, h2)

tensor([[[-0.0984,  0.4468,  0.9055,  ..., -0.0425,  0.9711,  0.1457]],

        [[-0.6505, -0.4054, -0.9629,  ...,  0.9957,  0.9785,  0.9475]],

        [[ 0.9774,  0.0373, -0.0968,  ...,  0.9955, -0.9904, -0.9782]],

        [[ 0.3751,  0.0068, -0.8749,  ..., -0.6759,  0.8896, -0.6378]]],
       device='cuda:0', grad_fn=<CudnnRnnBackward>) tensor([[[ -0.6777,   5.2808,  17.9910,  ...,  -0.2083,   5.5516,   0.3916]],

        [[ -2.1585,  -3.5053,  -5.5215,  ...,  44.2852,  18.6001,  11.6609]],

        [[ 24.8295,   1.1272,  -1.1426,  ...,  52.9754, -22.6928, -16.7860]],

        [[  3.4405,   0.0531,  -3.7376,  ...,  -2.8014,  10.2565,  -7.9510]]],
       device='cuda:0', grad_fn=<CudnnRnnBackward>)


#DATASET 2

In [ ]:
data2 = pd.read_csv("friends-personality.csv")

In [ ]:
print(data2.iloc[4,3])
data2.iloc[4,3] = 'c'
print(data2.iloc[4,3])

Monica Geller
c


In [ ]:
for i in range(len(data2)):
    if(data2.iloc[i][4] == 1):
        data2.iloc[i,3] = 'a'
    elif(data2.iloc[i][5] == 1):
        data2.iloc[i,3] = 'c'
    elif(data2.iloc[i][6] == 1):
        data2.iloc[i,3] = 'e'
    elif(data2.iloc[i][7] == 1):
        data2.iloc[i,3] = 'o'
    elif(data2.iloc[i][8] == 1):
        data2.iloc[i,3] = 'n'

data2.sample(frac=1) 
data2.head()


,Unnamed: 0,scene_id,text,character,cAGR,cCON,cEXT,cOPN,cNEU
0,0,01_e01_c01,<b>s01_e01_c01(1) for Joey Tribbiani</b><br><b...,a,1,1,0,0,1
1,1,01_e01_c02,<b>s01_e01_c02(0) for Chandler Bing</b><br><br...,a,1,0,1,1,1
2,2,01_e01_c04,<b>s01_e01_c04(0) for Chandler Bing</b><br><br...,n,0,0,0,0,1
3,3,01_e01_c05,<b>s01_e01_c05(0) for Paul</b><br><br><b>Monic...,a,1,0,1,1,0
4,4,01_e01_c08,<b>s01_e01_c08(0) for Monica Geller</b><br><br...,a,1,0,1,1,1


In [ ]:
datat2 = data2[571:]
print(datat2)

     Unnamed: 0    scene_id  ... cOPN cNEU
571         571  04_e10_c03  ...    1    1
572         572  04_e10_c08  ...    1    1
573         573  04_e10_c13  ...    1    1
574         574  04_e10_c13  ...    1    1
575         575  04_e10_c14  ...    0    0
..          ...         ...  ...  ...  ...
706         706  04_e24_c18  ...    0    0
707         707  04_e24_c19  ...    1    0
708         708  04_e24_c19  ...    1    0
709         709  04_e24_c20  ...    0    1
710         710  04_e24_c24  ...    1    1

[140 rows x 9 columns]


In [ ]:
datat2 = data2[:570]
print(datat2)

     Unnamed: 0    scene_id  ... cOPN cNEU
0             0  01_e01_c01  ...    0    1
1             1  01_e01_c02  ...    1    1
2             2  01_e01_c04  ...    0    1
3             3  01_e01_c05  ...    1    0
4             4  01_e01_c08  ...    1    1
..          ...         ...  ...  ...  ...
565         565  04_e09_c05  ...    1    0
566         566  04_e09_c09  ...    0    1
567         567  04_e09_c13  ...    1    0
568         568  04_e10_c02  ...    1    0
569         569  04_e10_c02  ...    0    0

[570 rows x 9 columns]


#LSTM 2

In [ ]:
def train2(inp, target, h1, h2, pretrain=0):
    if pretrain == 0:
        hidden = rnn.init_hidden()
        hidden2 = rnn.init_hidden()
    else:
        hidden = h1
        hidden = h2
        
    rnn.zero_grad()
    loss = 0

    inp = inp.cuda()
    hidden = hidden.cuda()
    hidden2 = hidden2.cuda()
    for c in range(chunk_len):
        if(inp[c] != 79 or inp[c] != 74):
            output, hidden, hidden2 = rnn(inp[c], hidden, hidden2)
            loss += loss_metric(output, target[0].unsqueeze(0))
        else:
            break

    loss.backward()
    optimizer.step()

    return loss.data.item() / chunk_len

In [ ]:
print(data2.iloc[23][2])

<b>s01_e04_c02(0) for Monica Geller</b><br><br><b>Monica Geller</b>: How does she do that?<br><br><b>Ross Geller</b>: I cannot sleep in a public place.<br><br><b>Monica Geller</b>: Would you look at her? She is so peaceful.<br><br><b>Phoebe Buffay</b>: (waking and startling them) Oh! What what what! ...Hi.<br><br><b>Ross Geller</b>: It's okay, y'know, you just nodded off again.<br><br><b>Monica Geller</b>: What's going on with you?<br><br><b>Phoebe Buffay</b>: I got no sleep last night!<br><br><b>Ross Geller</b>: Why?<br><br><b>Phoebe Buffay</b>: My grandmother has this new boyfriend, and they're both kind of insecure in bed. Oh, and deaf. So they're constantly, like, having to reassure each other that they're having a good time. You have no idea how loud they are!<br><br><b>Monica Geller</b>: Well, if you want, you can stay with Rachel and me tonight.<br><br>


In [ ]:
chunk_len = 150

# Gera um pedaço aleatório de texto com o tamanho especificado em chuck_len
def random_chunk2():
    index = random.randint(0, len(data2)-1)
    
    data = data2.iloc[index][2]
    
    for i in range(200-len(data)):#normaliza os chunks com tamanho igual
        data+='/0'
        
    return data, index
    
print(random_chunk2())

("<b>s04_e24_c13(0) for Phoebe</b><br><br><b>Joey</b>: Hello?<br><br><b>Phoebe</b>: (Angrily.) Hey, were the hell have you been?!<br><br><b>Joey</b>: Hey. I spent the night out. I met this cute bridesmaid. She is so...<br><br><b>Phoebe</b>: I don't want to hear about her!!<br><br><b>Joey</b>: Ahh Pheebs, you know you're still my number one girl.<br><br><b>Phoebe</b>: No! No, we have an emergency. Okay? Rachel's coming to London.<br><br><b>Joey</b>: Ohh great!!!<br><br><b>Phoebe</b>: No it's not great. No, she's coming to tell Ross that she loves him.<br><br><b>Joey</b>: (Confused.) But, he loves Emily?<br><br><b>Phoebe</b>: I KNOW THAT!!! You have to stop her!! She's going to ruin the wedding!!<br><br><b>Joey</b>: Okay.<br><br>", 700)


In [ ]:
chunk, index = random_chunk2()
print(chunk)

<b>s02_e24_c04(0) for Richard</b><br><br><b>Monica Geller</b>: (holding up a blanket) Where's Benny? (drops the blanket) There he is! (does it again) Where's Benny, there he is.<br><br><b>Richard</b>: Awww! You know that's probably why babies learn to talk, so they can tell grown ups to cut it out.<br><br><b>Monica Geller</b>: Hey, you know I got a question for ya. Just a little thing, no pressure.<br><br><b>Richard</b>: Okay.<br><br><b>Monica Geller</b>: Did you ever, uh, like, think about the future?<br><br><b>Richard</b>: Sure I do.<br><br><b>Monica Geller</b>: Yeah, am I in it?<br><br><b>Richard</b>: Honey, you are in it.<br><br><b>Monica Geller</b>: Oh God, you are about to get sooo lucky.<br><br><b>Richard</b>: Oh, yeah!<br><br><b>Monica Geller</b>: Keep talkin'.<br><br><b>Richard</b>: Well, uh, sometimes I think about selling my practice, we could move to France, make French toast.<br><br><b>Monica Geller</b>: Okay, so, uh, we're in France, we're making the toast. Do you see a l

In [ ]:
def random_training_set2():    
    chunk, index = random_chunk2()
    inp = char_tensor2(chunk)
    target = data2.iloc[index][3]
    target = char_tensor2(target)
    return inp, target

print(random_training_set2())

(tensor([63, 73, 25,  ..., 73, 34, 25]), tensor([83]))


In [ ]:
n_epochs = 100
print_every = 1
hidden_size = 512
n_layers = 4
lr = 0.0005

cuda = torch.device('cuda')

rnn = RNN(n_characters, hidden_size, n_characters, n_layers)
rnn.cuda()


optimizer = torch.optim.Adam(rnn.parameters(), lr=lr)
loss_metric = nn.CrossEntropyLoss()
h1 = 0
h2 = 0
start = time.time()
all_losses = []
loss_avg = 0
for epoch in range(1, n_epochs + 1):
    example = random_training_set2()
    #print(example[0])
    if i == 1:
        loss = train2(example[0].cuda(),example[1].cuda(), h1, h2, 1)
    else:
        loss = train2(example[0].cuda(),example[1].cuda(), h1, h2, 0)       
    loss_avg += loss

    if epoch % print_every == 0:
        print('[%s (%d %d%%) %.4f]' % (time_since(start), epoch, epoch / n_epochs * 100, loss))
       # print(evaluate('The', 100), '\n')

 0s (1 1%) 4.4607]
 0s (2 2%) 4.4661]
 0s (3 3%) 4.4909]
 1s (4 4%) 4.3434]
 1s (5 5%) 4.2101]
 1s (6 6%) 4.2032]
 2s (7 7%) 3.5066]
 2s (8 8%) 3.0426]
 2s (9 9%) 2.3719]
 3s (10 10%) 1.6188]
 3s (11 11%) 0.8923]
 3s (12 12%) 3.1209]
 3s (13 13%) 0.3072]
 4s (14 14%) 0.2353]
 4s (15 15%) 2.9412]
 4s (16 16%) 0.1935]
 5s (17 17%) 0.2044]
 5s (18 18%) 0.2115]
 5s (19 19%) 8.5337]
 6s (20 20%) 0.2338]
 6s (21 21%) 4.4551]
 6s (22 22%) 7.8094]
 6s (23 23%) 0.3237]
 7s (24 24%) 7.0873]
 7s (25 25%) 0.3916]
 7s (26 26%) 1.3865]
 8s (27 27%) 3.3068]
 8s (28 28%) 0.5516]
 8s (29 28%) 2.8486]
 9s (30 30%) 0.6532]
 9s (31 31%) 7.0841]
 9s (32 32%) 2.1145]
 10s (33 33%) 1.2221]
 10s (34 34%) 1.2092]
 10s (35 35%) 1.5068]
 10s (36 36%) 1.1396]
 11s (37 37%) 1.2274]
 11s (38 38%) 5.9018]
 11s (39 39%) 1.2861]
 12s (40 40%) 1.2648]
 12s (41 41%) 1.3960]
 12s (42 42%) 1.1690]
 13s (43 43%) 1.4254]
 13s (44 44%) 1.1689]
 13s (45 45%) 1.3780]
 13s (46 46%) 1.0589]
 14s (47 47%) 1.0363]
 14s (48 48%) 3.

#TEST

In [ ]:
file = open('friends-personality.csv').read()
all_characters2 = ''.join(set(file))
n_characters2 = len(all_characters2)
print(all_characters2)
print(n_characters2)
file_len = len(file)
print('file_len =', file_len)

6ztL4vuP,#xER[]nl$VWK?J%o>i&QjsghUrDH(NwT'`cX/YId3pqGkmZ0M1F:f5<2)
987;!-b_ Ca".OyBeS^A
87
file_len = 748760


In [ ]:
def char_tensor2(string):
    tensor = torch.zeros(len(string)).long()
    for c in range(len(string)):
        try:
            tensor[c] = all_characters2.index(string[c])
        except:
            print(c)
            raise
    return Variable(tensor)

print(char_tensor2('abcDEF'))

tensor([77, 73, 43, 35, 11, 59])


In [ ]:
# Gera um texto iniciando com "prime_str" de tamanho "predict_len" e variação definida por "temperature"
def evaluate2(prime_str='A', predict_len=1, temperature=0.8):
    hidden = rnn.init_hidden().cuda()
    hidden2 = rnn.init_hidden().cuda()
    prime_input = char_tensor2(prime_str).cuda()
    predicted = ''

    # prime_str é o texto inicial que o gerado irá completar
    for p in range(len(prime_str) - 1):
        _, hidden, hidden2 = rnn(prime_input[p], hidden, hidden2)
    inp = prime_input[-1]
    
    for p in range(predict_len):
        output, hidden, hidden2 = rnn(inp, hidden, hidden2)
        
        # Usa a temperatura para amostrar a distribuição e escolher a saída probabilísticamente
        output_dist = output.data.view(-1).div(temperature).exp()
        top_i = torch.multinomial(output_dist, 1)[0]
        
        # Adiciona o caracter predito à string de saída
        predicted_char = all_characters2[top_i]
        predicted += predicted_char
        inp = char_tensor2(predicted_char).cuda()

    return predicted

In [ ]:
#usando o evaluate para avaliar o modelo com o dataset de test
datat2 = datat2[datat2['text'].map(len) > 15]

correct = 0
loss = 0

for i in range(len(datat2)):
    predict = evaluate2(datat2.iloc[i][2])
    label = datat2.iloc[i][3]
    #print(predict, label)
    if(predict == label):
        correct+=1
    
        
        
        
accuracy = correct/(len(datat2))



print('Acurácia: %d%%'  % (accuracy * 100))

Acurácia: 42%
